In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 1367, done.
remote: Counting objects: 100% (216/216), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 1367 (delta 157), reused 193 (delta 143), pack-reused 1151
Receiving objects: 100% (1367/1367), 78.56 MiB | 18.80 MiB/s, done.
Resolving deltas: 100% (987/987), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 24.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorization_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorization_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorization_Cython_Epoch.c: In function ‘__pyx_pf_32MatrixFactorization

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps
from concurrent.futures import ThreadPoolExecutor

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv


In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid_linear_comb.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)
URM_validation, URM_test = split_train_in_two_percentage_global_sample(URM_test_complete, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.26348679053501006, 'beta': 0.17536156577039744, 'topK': 27}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'False',
 'implicit': 'False',
 'alpha': 0.3185935970252052,
 'topK': 48}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/pure_svd_params.json', 'r') as params_file:
    params_pure_svd = json.load(params_file)
params_pure_svd

{'factors': 30}

In [19]:
# with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_puresvd_item.json', 'r') as params_file:
#     params_pure_svd_item = json.load(params_file)
params_pure_svd_item = {"topK": 43, "num_factors": 171}

In [20]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_train)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_train)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_train)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_train_validation.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_train)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_train_validation.zip')

recommender_ease_r = EASE_R_Recommender(URM_train)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4636.45 column/sec. Elapsed time 4.82 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4642.05 column/sec. Elapsed time 2.81 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2646.98 column/sec. Elapsed time 8.44 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 3024.76 column/sec. Elapsed time 7.39 sec
PureSVDRecommender: URM Detected 627 ( 4.8%) users with no inter

In [21]:
weights = [0.4, 0.4, 0.2]
linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Linear Combinator')
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)
print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('User KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)
print('NMF')
result_df, _ = evaluator.evaluateRecommender(recommender_nmf)
display(result_df)
print('Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd)
display(result_df)
print('Item Pure SVD')
result_df, _ = evaluator.evaluateRecommender(recommender_pure_svd_item)
display(result_df)
print('SLIM BPR')
result_df, _ = evaluator.evaluateRecommender(recommender_slim_bpr)
display(result_df)
print('EASE R')
result_df, _ = evaluator.evaluateRecommender(recommender_ease_r)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Linear Combinator
EvaluatorHoldout: Processed 10456 (100.0%) in 16.72 sec. Users per second: 625


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096815,0.165347,0.139133,0.050065,0.085329,0.273897,0.152481,0.114179,0.51865,0.365354,...,0.802826,0.416385,0.802826,0.081728,10.631505,0.998377,0.235819,0.81901,1.535415,0.308298


Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 8.80 sec. Users per second: 1188


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097102,0.165756,0.139351,0.050173,0.085482,0.27428,0.152762,0.114452,0.520658,0.365828,...,0.802826,0.417998,0.802826,0.087463,10.729146,0.998493,0.252365,0.826532,1.477653,0.310478


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 8.69 sec. Users per second: 1203


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.065599,0.109738,0.091334,0.033326,0.055165,0.209391,0.104188,0.076356,0.402831,0.262119,...,0.802826,0.323403,0.802826,0.019058,9.121761,0.997687,0.054991,0.702705,1.408899,0.291045


SLIM BPR
EvaluatorHoldout: Processed 10456 (100.0%) in 10.17 sec. Users per second: 1029


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.054524,0.094253,0.079452,0.0259,0.044669,0.176272,0.086926,0.064669,0.36754,0.212465,...,0.802826,0.295071,0.802826,0.034099,9.527262,0.996677,0.098389,0.733943,2.03833,0.294086


EASE R
EvaluatorHoldout: Processed 10456 (100.0%) in 14.04 sec. Users per second: 745


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.074857,0.135323,0.116287,0.037511,0.067475,0.233553,0.123573,0.091082,0.459353,0.293884,...,0.802826,0.368781,0.802826,0.019069,8.350538,0.991242,0.055022,0.643293,3.089786,0.267475


In [22]:
weights = [0.5, 0.8, 0.8, 0.05, 0.01, 0.005, 0.005, 0.005]
recommenders = [recommender_item, recommender_rp3, recommender_p3a, recommender_ease_r, recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr]
linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
result_df, _ = evaluator.evaluateRecommender(linear_combinator)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 38.02 sec. Users per second: 275


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097972,0.169844,0.143603,0.05047,0.086782,0.277102,0.15512,0.116478,0.525536,0.368998,...,0.802826,0.421913,0.802826,0.067755,10.310941,0.997682,0.195499,0.794315,1.813323,0.300063


In [23]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

def objective(trial):
    weights = [
        trial.suggest_float('item_knn', 1e-2, 1, log=True),
        trial.suggest_float('rp3', 1e-2, 1, log=True),
        trial.suggest_float('p3alpha', 1e-2, 1, log=True),
        trial.suggest_float('ease_r', 1e-3, 1, log=True),
        trial.suggest_float('user_knn', 1e-3, 1, log=True),
        trial.suggest_float('svd_item', 1e-4, 1, log=True),
        trial.suggest_float('nmf', 1e-5, 1, log=True),
        trial.suggest_float('slim_bpr', 1e-5, 1, log=True)
    ]
    
    recommenders = [
        recommender_item, recommender_rp3, recommender_p3a, recommender_ease_r, 
        recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
    ]
    
    linear_combinator = LinearCombinationRecommender(URM_train, recommenders, weights)
    evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])
    result_df, _ = evaluator.evaluateRecommender(linear_combinator)

    return result_df.loc[10, 'MAP']

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions


In [24]:
# def evaluate_linear_model(URM_train, URM_valid, weights):
#     recommender_item = ItemKNNCFRecommender(URM_train)
#     recommender_item.fit(**params_item_knn)

#     recommender_rp3 = RP3betaRecommender(URM_train)
#     recommender_rp3.fit(**params_rp3)
    
#     recommender_p3a = P3alphaRecommender(URM_train)
#     recommender_p3a.fit(**params_p3alpha)
    
#     linear_combinator = LinearCombinationRecommender(URM_train, [recommender_item, recommender_rp3, recommender_p3a], weights)    
#     evaluator = EvaluatorHoldout(URM_valid, cutoff_list=[10])
#     result_df, _ = evaluator.evaluateRecommender(linear_combinator)
#     return result_df.loc[10, 'MAP']

# def objective(trial):
#     w1 = trial.suggest_float('w1', 0.01, 0.99)
#     w2 = trial.suggest_float('w2', 0.01, 0.99)
#     w3 = trial.suggest_float('w3', 0.01, 0.99)
    
#     results = [None] * 10
#     with ThreadPoolExecutor(max_workers=10) as executor:
#         futures = []
#         for URM_train, URM_valid in CrossValidationSplitter(10).split(URM_all):
#             futures.append(
#                 executor.submit(evaluate_linear_model, URM_train, URM_valid, [w1, w2, w3])
#             )
#         for i, future in enumerate(futures):
#             results[i] = future.result()
#     return np.mean(results)

In [25]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid_merging.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass

In [26]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_linear_8models_no_cv', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=100)

[I 2023-12-01 18:07:26,923] Using an existing study with name 'hyperparameters_tuning_linear_8models_no_cv' instead of creating a new one.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 38.02 sec. Users per second: 275


[I 2023-12-01 18:08:05,360] Trial 100 finished with value: 0.05024663821789943 and parameters: {'item_knn': 0.03240293348185069, 'rp3': 0.3989278335011647, 'p3alpha': 0.16576975301216076, 'ease_r': 0.001975268855694896, 'user_knn': 0.007789214219038378, 'svd_item': 0.0019218832242984673, 'nmf': 0.0012948162773743946, 'slim_bpr': 4.721274530606856e-05}. Best is trial 93 with value: 0.050696577343001106.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 37.91 sec. Users per second: 276


[I 2023-12-01 18:08:43,541] Trial 101 finished with value: 0.050712141430150896 and parameters: {'item_knn': 0.04986228232741684, 'rp3': 0.3433816004027039, 'p3alpha': 0.19062903201076378, 'ease_r': 0.001231644222265787, 'user_knn': 0.0036696132974807175, 'svd_item': 0.003060739381027495, 'nmf': 0.0017977262444814504, 'slim_bpr': 8.573146591938293e-05}. Best is trial 101 with value: 0.050712141430150896.


...

[I 2023-12-01 19:10:03,240] Trial 197 finished with value: 0.05071672222829518 and parameters: {'item_knn': 0.03799142368862948, 'rp3': 0.2626655484493735, 'p3alpha': 0.15246295042694752, 'ease_r': 0.003484412852365707, 'user_knn': 0.003315662868508929, 'svd_item': 0.0015738416183407188, 'nmf': 0.0008534812841641027, 'slim_bpr': 9.712015521895892e-05}. Best is trial 102 with value: 0.050744833224032365.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 38.06 sec. Users per second: 275


[I 2023-12-01 19:10:41,581] Trial 198 finished with value: 0.050711397572291456 and parameters: {'item_knn': 0.038750804368770236, 'rp3': 0.21553220612956253, 'p3alpha': 0.13631523255237815, 'ease_r': 0.003250752601226972, 'user_knn': 0.003163998437783083, 'svd_item': 0.0015665393542817633, 'nmf': 0.0009022078759328296, 'slim_bpr': 0.00011721877330504543}. Best is trial 102 with value: 0.050744833224032365.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
EvaluatorHoldout: Processed 10456 (100.0%) in 38.19 sec. Users per second: 274


[I 2023-12-01 19:11:20,060] Trial 199 finished with value: 0.05072470351951089 and parameters: {'item_knn': 0.034884989254947936, 'rp3': 0.21192268076786414, 'p3alpha': 0.12592170825815036, 'ease_r': 0.0034508474586898985, 'user_knn': 0.0022360570158350703, 'svd_item': 0.001185382713830819, 'nmf': 0.0008372247262286593, 'slim_bpr': 0.00012393762627272217}. Best is trial 102 with value: 0.050744833224032365.


In [27]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_param_importances(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [30]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_hybrid_lr_comb_8models.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_hybrid_lr_comb_8models.json', 
            'tuning_results/best_params_hybrid_lr_comb_8models.json', 
            'Linear combination of 8 models tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/linear-combination/best_params_hybrid_lr_comb_8models.json', 
        '/kaggle/working/best_params_hybrid_lr_comb_8models.json'
    )

In [31]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'],
        'tuning_results/tuning_hybrid_merging.db', 
        'Linear combination of Item KNN, RP3 and P3alpha tuning db updated results (from kaggle notebook)'
    )

In [32]:
with open('/kaggle/working/best_params_hybrid_lr_comb_8models.json', 'r') as params_file:
    params = json.load(params_file)
params

{'item_knn': 0.050774640152222925,
 'rp3': 0.35730087685145995,
 'p3alpha': 0.18757329883116283,
 'ease_r': 0.0013286016102773412,
 'user_knn': 0.0036521844735514794,
 'svd_item': 0.0033963853363314905,
 'nmf': 0.0017799916269743072,
 'slim_bpr': 6.43751837916623e-05}

In [33]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_pure_svd = PureSVDRecommender(URM_all)
recommender_pure_svd.fit(num_factors=params_pure_svd['factors'])

recommender_pure_svd_item = PureSVDItemRecommender(URM_all)
recommender_pure_svd_item.fit(**params_pure_svd_item)

recommender_nmf = NMFRecommender(URM_all)
recommender_nmf.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'nmf_all.zip')

recommender_slim_bpr = SLIM_BPR_Python(URM_all)
recommender_slim_bpr.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_bpr_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4279.61 column/sec. Elapsed time 5.22 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 4106.46 column/sec. Elapsed time 3.17 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2449.63 column/sec. Elapsed time 9.12 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2750.64 column/sec. Elapsed time 8.12 sec
PureSVDRecommender: URM Detected 386 ( 3.0%) users with no inter

In [34]:
weights = [
    params['item_knn'],
    params['rp3'],
    params['p3alpha'],
    params['ease_r'],
    params['user_knn'],
    params['svd_item'],
    params['nmf'],
    params['slim_bpr']
]

recommenders = [
    recommender_item, recommender_rp3, recommender_p3a, recommender_ease_r, 
    recommender_user, recommender_pure_svd_item, recommender_nmf, recommender_slim_bpr
]

In [35]:
linear_combinator = LinearCombinationRecommender(URM_all, recommenders, weights)
recommendations = linear_combinator.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())

submission_df.to_csv('/kaggle/working/submission_linear_comb_knn_rp3_p3a.csv', index=False)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


,user_id,item_list
0,1,36 101 403 506 123 515 694 254 139 1546
1,2,1095 47 28 11 50 12 1522 196 102 656
2,3,59 259 239 584 536 857 956 511 648 414
3,4,28 249 50 139 7 254 171 639 314 129
4,5,1570 77 5138 116 131 148 4 95 7033 766
